In [1013]:
import pandas as pd
from pandas import read_csv
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

In [1014]:
X_orig = read_csv('AML/task1/X_train.csv',',')
Y_orig = read_csv('AML/task1/y_train.csv',',')
#X_train, X_test, Y_train, Y_test = train_test_split(X_orig, Y_orig, test_size=0.20, random_state=10, shuffle=True)
X_train=X_orig
Y_train=Y_orig
X_test = read_csv('AML/task1/X_test.csv')
X_train=X_train.set_index('id')
X_test=X_test.set_index('id')
Y_train=Y_train.set_index('id')
#Y_test=Y_test.set_index('id')
#Y_test=Y_test.set_index('id')

In [1015]:
imp = Imputer(missing_values=np.NaN, strategy="median", axis=0)

In [1016]:
X_train_id = X_train.index
X_train = imp.fit_transform(X_train)
X_test_id = X_test.index
X_test = imp.transform(X_test)

In [1017]:
#print(X_train.shape)
iso = IsolationForest( n_estimators=20, max_samples='auto', contamination=float(0.005), max_features=20)
iso.fit(X_train)

IsolationForest(bootstrap=False, contamination=0.005, max_features=20,
        max_samples='auto', n_estimators=20, n_jobs=1, random_state=None,
        verbose=0)

In [1018]:
##### Outlier detection in X_train
yhat = iso.predict(X_train)
mask = yhat != -1 #this mask tells if the output is an outlier or not
X_train = pd.DataFrame(X_train)
X_train['id'] = X_train_id
X_train=X_train.set_index('id')

N=X_train.shape[0]
for i in range(0,N):
    if(mask[i]==False):
        X_train=X_train.drop(X_train_id[i])
        Y_train=Y_train.drop(X_train_id[i])
        print("Outlier in train is i = "+ str(i), flush=True)
#Now the outliers have been filtered out for X_train

Outlier in train is i = 87
Outlier in train is i = 363
Outlier in train is i = 379
Outlier in train is i = 591
Outlier in train is i = 780
Outlier in train is i = 794
Outlier in train is i = 1008


In [1019]:
##Outlier detection in X_test
#yhat = iso.predict(X_test)
#mask = yhat != -1 #this mask tells if the output is an outlier or not
#X_test = pd.DataFrame(X_test)
#X_test['id'] = X_test_id
#X_test=X_test.set_index('id')

#N=X_test.shape[0]
#for i in range(0,N):
#    if(mask[i]==False):
#        print("Outlier in test is i = " + str(i), flush = True)
#        X_test=X_test.drop(X_test_id[i])
#        #Y_test=Y_test.drop(X_test_id[i])
##Now the outliers have been filtered out for X_test

In [1020]:
## Feature selection https://machinelearningmastery.com/feature-selection-for-regression-data/
#fs = SelectKBest(score_func=f_regression, k=400)
fs = SelectKBest(score_func=f_regression, k=400)
Y=Y_train.T
Y=Y.values.flatten()
fs.fit(X_train, Y)
X_train = fs.transform(X_train)
X_test = fs.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/feature_selection/univariate_selection.py:292: RuntimeWarning: invalid value encountered in sqrt
  n_samples * X_means ** 2)
/usr/lib/python3/dist-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/lib/python3/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/lib/python3/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/lib/python3/dist-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [1021]:
# Train data
#model = LinearRegression()
#model.fit(X_train, Y_train)
#Y_predict = model.predict(X_train)
estimator = RandomForestRegressor(n_estimators=50, max_depth = 20,random_state = 1)
selector = RFE(estimator, n_features_to_select=120, step=40, verbose = 1)
selector = selector.fit(X_train, Y)
Y_predict = selector.predict(X_train)
print(r2_score(Y_train,Y_predict))
Y_predict = selector.predict(X_test)
#print(r2_score(Y_test,Y_predict))
Y_df = pd.DataFrame(Y_predict)
Y_df=Y_df.reset_index()
Y_df.columns=['id','y']
Y_df.to_csv("NewTest.csv",header=['id','y'])

Fitting estimator with 400 features.
Fitting estimator with 360 features.
Fitting estimator with 320 features.
Fitting estimator with 280 features.
Fitting estimator with 240 features.
Fitting estimator with 200 features.
Fitting estimator with 160 features.
0.940026943199


In [1022]:
#regr = make_pipeline(StandardScaler(), SVR(C=1, epsilon=0.1))

In [1023]:
#Y=Y.values.flatten()
#regr.fit(X, Y)
#Y_predict=regr.predict(X)
#score = r2_score(Y,Y_predict)
#print(score)

In [1024]:
#lda = LDA(n_components=200)
#X = lda.fit_transform(X, Y)
#X_test = lda.transform(X_test)

In [1025]:
#X.shape

In [1026]:
#estimator = RandomForestRegressor(n_estimators=250, max_depth = 15,random_state = 1)
#selector = RFE(estimator, n_features_to_select=120, step=10, verbose = 1)
#selector = RFECV(estimator, cv=3, min_features_to_select = 150, step = 0.15, n_jobs = 4, verbose = 1)
#selector = RFECV(estimator, cv=3, step = 0.15, n_jobs = 4, n_features_to_select=120, verbose = 1)
#selector = selector.fit(X, Y)
#Y_predict = selector.predict(X)
#score = r2_score(Y,Y_predict)
#print(score)

In [1027]:
#X.shape

In [1028]:
#dir(selector)
#regr.fit(X, Y)

In [1029]:
#Y_predict=regr.predict(X)
#score = r2_score(Y,Y_predict)
#print(score)

In [1030]:
#clf = make_pipeline(StandardScaler(), SVC(gamma='auto', C=1, tol=0.1))
#clf.fit(X, Y)
#Y_predict=clf.predict(X)
#score = r2_score(Y,Y_predict)
#print(score)

In [1031]:
#Y_test_new=selector.predict(X_test)
#Y_test=Y_test.T
#Y_test=Y_test.values.flatten()
#score=r2_score(Y_test_new, Y_test)
#Y_df = pd.DataFrame(Y_test_new)
#Y_df=Y_df.reset_index()
#Y_df.columns=['id','y']

In [1032]:
#Y_df.to_csv("RandomTest.csv",header=['id','y'])